In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [22]:
device = "cpu"

In [23]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [24]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


# No Line search

In [26]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.GradientDescentLS(model=model, lr=0.005, line_search_method="const")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.7050851583480835
epoch:  1, loss: 0.6846209168434143
epoch:  2, loss: 0.6648592948913574
epoch:  3, loss: 0.6457696557044983
epoch:  4, loss: 0.6273196339607239
epoch:  5, loss: 0.6094874143600464
epoch:  6, loss: 0.5922489762306213
epoch:  7, loss: 0.5755816698074341
epoch:  8, loss: 0.5594626069068909
epoch:  9, loss: 0.5438734292984009
epoch:  10, loss: 0.5287901163101196
epoch:  11, loss: 0.5141943097114563
epoch:  12, loss: 0.5000653862953186
epoch:  13, loss: 0.4863907992839813
epoch:  14, loss: 0.47315213084220886
epoch:  15, loss: 0.4603339433670044
epoch:  16, loss: 0.4479227066040039
epoch:  17, loss: 0.4359026849269867
epoch:  18, loss: 0.42426154017448425
epoch:  19, loss: 0.41298922896385193
epoch:  20, loss: 0.40206992626190186
epoch:  21, loss: 0.3914913833141327
epoch:  22, loss: 0.38124287128448486
epoch:  23, loss: 0.3713095486164093
epoch:  24, loss: 0.36169540882110596
epoch:  25, loss: 0.352390855550766
epoch:  26, loss: 0.3433999717235565
epoch:

In [27]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = -12325.920207410292
Test metrics:  R2 = -10845.116244829333


# Backtrack line search

In [28]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.11204637587070465
epoch:  1, loss: 0.08480273932218552
epoch:  2, loss: 0.06693968176841736
epoch:  3, loss: 0.05539634823799133
epoch:  4, loss: 0.04845454543828964
epoch:  5, loss: 0.04429106414318085
epoch:  6, loss: 0.04179423674941063
epoch:  7, loss: 0.04029738903045654
epoch:  8, loss: 0.03940034285187721
epoch:  9, loss: 0.03886290639638901
epoch:  10, loss: 0.03854095935821533
epoch:  11, loss: 0.03834807127714157
epoch:  12, loss: 0.03823243826627731
epoch:  13, loss: 0.03816304728388786
epoch:  14, loss: 0.03812132403254509
epoch:  15, loss: 0.038096148520708084
epoch:  16, loss: 0.03808087483048439
epoch:  17, loss: 0.03807152062654495
epoch:  18, loss: 0.03806570917367935
epoch:  19, loss: 0.038064900785684586
epoch:  20, loss: 0.03805931285023689
epoch:  21, loss: 0.038058456033468246
epoch:  22, loss: 0.038053084164857864
epoch:  23, loss: 0.03805212676525116
epoch:  24, loss: 0.03804696723818779
epoch:  25, loss: 0.03804595395922661
epoch:  26, loss: 

In [16]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.7932532270998592
Test metrics:  R2 = 0.7384289812147724


## Bisection line search

In [29]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.GradientDescentLS(model=model, lr=1, line_search_method="bisect")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.16624541580677032
epoch:  1, loss: 0.04118894040584564
epoch:  2, loss: 0.0379139669239521
epoch:  3, loss: 0.03787641227245331
epoch:  4, loss: 0.03783929720520973
epoch:  5, loss: 0.037801891565322876
epoch:  6, loss: 0.03776330128312111
epoch:  7, loss: 0.03772393614053726
epoch:  8, loss: 0.0376836434006691
epoch:  9, loss: 0.037623774260282516
epoch:  10, loss: 0.03755364567041397
epoch:  11, loss: 0.03734752535820007
epoch:  12, loss: 0.03718986362218857
epoch:  13, loss: 0.0371243841946125
epoch:  14, loss: 0.03708893433213234
epoch:  15, loss: 0.037007030099630356
epoch:  16, loss: 0.03692341968417168
epoch:  17, loss: 0.03684909641742706
epoch:  18, loss: 0.03681043162941933
epoch:  19, loss: 0.036716751754283905
epoch:  20, loss: 0.036620527505874634
epoch:  21, loss: 0.0365365706384182
epoch:  22, loss: 0.03648823872208595
epoch:  23, loss: 0.036377716809511185
epoch:  24, loss: 0.03626425564289093
epoch:  25, loss: 0.036181915551424026
epoch:  26, loss: 0

In [30]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.7597976717455035
Test metrics:  R2 = 0.7406935411879965
